In [ ]:
!nvidia-smi

Thu Mar 14 01:12:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.


In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below is a interview of elon musk. Mimic the talking style of elon musk. Generate sentences that sound like coming from elon musk.
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Response:
{response}
""".strip()

In [ ]:
question = "What's your favorite planet?"
answer = "Mars"
generate_training_prompt(question,answer)

"### Instruction: Below is a interview of elon musk. Mimic the talking style of elon musk. Generate sentences that sound like coming from elon musk.\n\n### Input:\nWhat's your favorite planet?\n\n### Response:\nMars"

In [ ]:
elon_file = "/content/elonmusk-ft.txt"
with open("/content/elonmusk-ft.txt","r") as tx:
  interview_text = tx.readlines()

In [ ]:
interview_text

['Elon Musk, Tesla and SpaceX chief executive, spoke to Peter Campbell, global motor industry correspondent, at FT Live’s Future of the Car conference in London\xa0on Tuesday May 10. This is an edited transcript of their interview, which covered everything from buying a mining company to lifting the Twitter ban on Donald Trump.\n',
 '\n',
 'Peter Campbell (PC): Let me ask you to put your product development hat on. And if you pare Twitter all the way back, what is it and what do you think it can be in 10 years’ time?\n',
 '\n',
 'Elon Musk (EM): Well, what I’ve said is that I think Twitter is currently the best, or looked at another way, the least bad public square, a forum for the exchange of ideas, nationally and internationally. But I think it could be a lot better at that. In order to be better at that, it needs to really get rid of the bots and the scams and spammers, and you know, basically, anyone [who] tried to create sort of fake influence on the site, whereby one person is op

In [ ]:
questions = []
answers = []
for sentence in interview_text[1:]:
  if sentence[0:2] == "PC":
    questions.append(sentence)
  elif sentence[0:2] == "EM":
    answers.append(sentence)


In [ ]:
questions

['PC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n',
 'PC: But that’s at the moment because of the global supply chains and the chip shortages. That’s less around electric car demand which we are expecting to go through the roof.\n',
 'PC: I have two more questions on Twitter if I may, before we turn for the rest of the session to Tesla. How confident are you the deal will happen, because there is a risk, because you are putting a lot of your personal stake up to fund it, that if the whole thing goes south, you’re imperilling your stake in Tesla, and potentially in SpaceX, if it all goes to pot. That’s the technical term.\n',
 'PC: And the final question, and this is really the toupe’d elephant in the room. Are you planning to let Donald Trump back on?\n',
 'PC: Ev

In [ ]:
answers

['EM: I’m confident that we will be able to sell all the cars we can make. I mean, currently, the lead time for ordering a Tesla is ridiculously long, so our issue is not demand, it is production.\n',
 'EM: Yeah, I mean even before there were supply chain issues, Tesla demand exceeded production. So now its demand is exceeding production to a ridiculous degree. We’re actually probably going to limit that. Just stop taking orders for anything beyond a certain period of time because some of the timing is like a year away. So anyway, the frustration that we’re seeing with customers is being unable to get them a car, not are they willing or interested in, buying a car? So basically, I think zero about demand generation and a lot about production, and engineering and supply chain.\n',
 'EM: Yeah, Sure. So, I mean, I think there’s still a lot of things that need to get done before this deal concludes. Obviously there’s not yet even been a shareholder vote and Twitter has not yet filed the pr

In [ ]:
list = [{"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)} for question,answer in zip(questions,answers)]
data = Dataset.from_list(list)



In [ ]:
list

[{'question': 'PC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n',
  'answer': 'EM: I’m confident that we will be able to sell all the cars we can make. I mean, currently, the lead time for ordering a Tesla is ridiculously long, so our issue is not demand, it is production.\n',
  'training_prompt': '### Instruction: Below is a interview of elon musk. Mimic the talking style of elon musk. Generate sentences that sound like coming from elon musk.\n\n### Input:\nPC: Because people will automatically associate you with Tesla and you with Twitter, is there any risk in your mind that the actions that you’re going to take at Twitter, which you’ve admitted will upset some people, will potentially lead to a commercial impact on Tesla.\n\n### Response:\nEM: I’m confident that

In [ ]:
notebook_login()

In [ ]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [ ]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [ ]:
model.config.quantization_config.to_dict()

NameError: name 'model' is not defined

In [ ]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
OUTPUT_DIR = "experiments"

%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    eval_dataset=data,
    peft_config=peft_config,
    dataset_text_field="training_prompt",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()